# LightGBM model with Airline dataset. Experiment 01

LightGBM regressor with L2 + Airline dataset

In [22]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm.sklearn import LGBMRegressor
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from scipy.stats import gmean

from libs.timer import Timer
from libs.loaders import load_fraud, load_iot, load_airline
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.conversion import _get_nominal_integer_dict, _convert_to_integer

print("System version: {}".format(sys.version))

System version: 3.6.0 |Anaconda 4.3.1 (64-bit)| (default, Dec 23 2016, 12:22:00) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


### Load data

In [2]:
%%time
df_plane = load_airline()
print(df_plane.shape)


MOUNT_POINT not found in environment. Defaulting to /fileshare
(115069017, 14)
CPU times: user 1min 40s, sys: 14.7 s, total: 1min 55s
Wall time: 3min 35s


In [3]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [4]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])


CPU times: user 2min 8s, sys: 7.82 s, total: 2min 16s
Wall time: 2min 13s


In [5]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [6]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')


CPU times: user 1min 8s, sys: 7.12 s, total: 1min 15s
Wall time: 1min 13s


In [7]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [8]:
#gets a random 80% of the entire set
X_train = df_plane_numeric.sample(frac=0.8, random_state=1)
print(X_train.shape)
#gets the left out portion of the dataset
X_test = df_plane_numeric.loc[~df_plane_numeric.index.isin(X_train.index)]
print(X_test.shape)

(92055214, 14)
(23013803, 14)


### Training

In [9]:
# Variables
KFOLD_SPLITS = 5

In [18]:
def train_lightgbm(df, verbose=True):
    if verbose: print("Training data of size {}".format(df.shape))
    x = df.ix[:, df.columns != 'ArrDelay'].as_matrix()
    y = df['ArrDelay'].as_matrix()
    
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, random_state=77, shuffle=True)
    clfs = []
    
    if verbose: print("Computing LightGBM boosted tree using {} kfold cross validation".format(n_splits))
    for train_index, test_index in skf.split(x, y):
        trn_x, val_x = x[train_index,:], x[test_index,:]
        trn_y, val_y = y[train_index], y[test_index]

        clf = LGBMRegressor(max_depth=6,
                            num_leaves=21,
                            n_estimators=300,
                            min_child_weight=30,
                            learning_rate=0.01,
                            nthread=20,
                            boosting_type='gbdt',
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=77)
        clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=verbose, eval_metric='l2', early_stopping_rounds=300)
        clfs.append(clf)
        
    return clfs

In [16]:
X_train_small = X_train.sample(frac=0.001, random_state=1)
print(X_train_small.shape)

(92055, 14)


In [ ]:
%%time
clfs = train_lightgbm(X_train)

### Evaluation

In [29]:
def compute_prediction(clfs, df, verbose=True):    
    x = df.as_matrix()
            
    preds = []
    with Timer() as t:
        for clf in clfs:
            preds.append(clf.predict(x))
        pred = gmean(np.array(preds), axis=0)
    if verbose: print("Prediction took %.03f sec.\n" % t.interval)        
    df['prediction'] = pred
    return df

In [27]:
X_test_small = X_test.sample(frac=0.001, random_state=1)
print(X_test_small.shape)

(23014, 14)


In [31]:
X_test_pred = compute_prediction(clfs, X_test)
X_test_pred.head(50)

Prediction took 6.027 sec.



/home/hoaphumanoid/anaconda3/envs/strata/lib/python3.6/site-packages/scipy/stats/stats.py:314: RuntimeWarning: invalid value encountered in log
  log_a = np.log(a)


,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,prediction
85782914,2004,9,8,3,636,709,18,6098,24,1,13,47,0,-14,NaN
33917016,1995,5,19,5,1755,2003,11,751,302,50,22,2075,0,8,12.583925
101255940,2006,11,23,4,1035,1255,0,664,177,109,72,1183,0,32,1.106149
93564630,2005,10,16,7,1122,1520,0,730,170,33,43,1182,0,-7,1.322369
16885574,1992,1,6,1,1709,1902,0,1119,220,10,0,1464,0,70,11.061411
13132449,1991,4,9,2,1104,1348,0,570,100,131,10,592,0,-6,1.500661
2477763,1988,3,26,6,2233,2331,6,831,49,62,19,270,0,-8,4.715773
90743270,2005,5,24,2,1145,1235,10,1407,50,90,94,189,0,0,0.569552
33857972,1995,5,15,1,1715,1745,11,1027,85,58,48,426,0,-4,6.998545
51966358,1998,10,30,5,1305,1426,4,2809,76,16,52,391,0,-8,4.304622
